In [11]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

def apply_clahe_to_image(image_path, output_path, clip_limit=2.0, tile_grid_size=(16, 16)):
    # 이미지 읽기 (이미지를 RGB로 읽음)
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: 이미지 파일을 읽을 수 없습니다: {image_path}")
        return False

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # OpenCV의 기본 BGR을 RGB로 변환

    # RGB에서 YUV 색 공간으로 변환
    img_yuv = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2YUV)

    # CLAHE 객체 생성 및 적용
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    img_yuv[:, :, 0] = clahe.apply(img_yuv[:, :, 0])  # Y 채널에 CLAHE 적용

    # 다시 RGB 색 공간으로 변환
    img_clahe_rgb = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2RGB)

    # CLAHE 적용 이미지를 파일로 저장
    success = cv2.imwrite(output_path, cv2.cvtColor(img_clahe_rgb, cv2.COLOR_RGB2BGR))
    if success:
        print(f"Saved processed image to: {output_path}")
    else:
        print(f"Error: Failed to save image to: {output_path}")
    return success

def has_class_1_label(label_path):
    with open(label_path, 'r') as label_file:
        labels = label_file.readlines()
        return any(' 1 ' in label or label.startswith('1 ') for label in labels)

# 폴더 경로 설정
input_images_folder = '/home/under1/Detect/jeongui/sagittal/base/images/train'
input_labels_folder = '/home/under1/Detect/jeongui/sagittal/base/labels/train'
output_images_folder = '/home/under1/Detect/jeongui/sagittal/CLAHE/CLAHE_positive/images/train'

# 출력 폴더가 존재하지 않으면 생성
os.makedirs(output_images_folder, exist_ok=True)

# 이미지 파일에 CLAHE 적용
processed_images = []
for root, dirs, files in os.walk(input_images_folder):
    for filename in files:
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            input_image_path = os.path.join(root, filename)
            relative_path = os.path.relpath(input_image_path, input_images_folder)
            output_image_path = os.path.join(output_images_folder, relative_path)
            
            # 대응하는 라벨 파일 경로
            label_path = os.path.join(input_labels_folder, os.path.splitext(relative_path)[0] + '.txt')
            
            # 라벨 파일이 존재하고 클래스 1인 객체가 포함된 경우에만 CLAHE 적용
            if os.path.exists(label_path) and has_class_1_label(label_path):
                os.makedirs(os.path.dirname(output_image_path), exist_ok=True)
                if apply_clahe_to_image(input_image_path, output_image_path):
                    processed_images.append(output_image_path)

Saved processed image to: /home/under1/Detect/jeongui/sagittal/CLAHE/CLAHE_positive/images/train/00071_259.jpg
Saved processed image to: /home/under1/Detect/jeongui/sagittal/CLAHE/CLAHE_positive/images/train/00059_122.jpg
Saved processed image to: /home/under1/Detect/jeongui/sagittal/CLAHE/CLAHE_positive/images/train/00177_125.jpg
Saved processed image to: /home/under1/Detect/jeongui/sagittal/CLAHE/CLAHE_positive/images/train/00128_243.jpg
Saved processed image to: /home/under1/Detect/jeongui/sagittal/CLAHE/CLAHE_positive/images/train/00138_217.jpg
Saved processed image to: /home/under1/Detect/jeongui/sagittal/CLAHE/CLAHE_positive/images/train/00060_72.jpg
Saved processed image to: /home/under1/Detect/jeongui/sagittal/CLAHE/CLAHE_positive/images/train/00074_249.jpg
Saved processed image to: /home/under1/Detect/jeongui/sagittal/CLAHE/CLAHE_positive/images/train/00159_189.jpg
Saved processed image to: /home/under1/Detect/jeongui/sagittal/CLAHE/CLAHE_positive/images/train/00177_221.jpg
Sa

In [15]:
# negative 이미지들 가져오기
import os
import shutil

def copy_images_based_on_labels(input_folder, label_folder, output_folder):
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)
    
    # Loop through all files in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):  # Check if the file is an image
            label_path = os.path.join(label_folder, os.path.splitext(filename)[0] + '.txt')
            
            # Check if the corresponding label file exists
            if os.path.exists(label_path):
                with open(label_path, 'r') as label_file:
                    labels = label_file.readlines()
                    
                    # Check if all labels have class 0
                    all_class_0 = all(label.startswith('0 ') for label in labels)
                    
                    if all_class_0:
                        # Copy the image to the output folder
                        shutil.copy(os.path.join(input_folder, filename), os.path.join(output_folder, filename))

# Usage example
input_folder = '../../base/images/train'
label_folder = '../../base/labels/train'
output_folder = './images/train'
copy_images_based_on_labels(input_folder, label_folder, output_folder)